In [1]:
import numpy as np
import pandas as pd

In [2]:
import projections
projector = projections.Projections()

## Define Matrices

## M_a

In [3]:
M_a = np.ones((4, 4)) * 0.01 + np.diag([8, 4, 1, 1])
M_a.round(decimals=3)

array([[8.01, 0.01, 0.01, 0.01],
       [0.01, 4.01, 0.01, 0.01],
       [0.01, 0.01, 1.01, 0.01],
       [0.01, 0.01, 0.01, 1.01]])

## M_b

In [4]:
g_ba = np.array([
    [1., 1, 0, 0],
    [0, 0, 1, 1]
])

In [5]:
Lambda_col_a, V_a = np.linalg.eig(M_a)
Lambda_col_a, V_a.round(decimals=3)

(array([8.01005376, 1.01990469, 4.01004155, 1.        ]),
 array([[ 1.   ,  0.002, -0.003, -0.   ],
        [ 0.003,  0.005,  1.   ,  0.   ],
        [ 0.001, -0.707,  0.003, -0.707],
        [ 0.001, -0.707,  0.003,  0.707]]))

In [6]:
n_b, n_a = g_ba.shape

idx = Lambda_col_a.argsort()[-n_b:][::-1]   # -n_b
Lambda_col_a_top_b = Lambda_col_a[idx]  # top 1 eigenvector... # n_b eigenvalues
V_a_top_b = V_a[:,idx]           # corresponding eigenvectors

Lambda_col_a_top_b, V_a_top_b.round(decimals=3)

(array([8.01005376, 4.01004155]),
 array([[ 1.   , -0.003],
        [ 0.003,  1.   ],
        [ 0.001,  0.003],
        [ 0.001,  0.003]]))

In [7]:
A = np.array([
    [-1, 2],
    [2, -1.]
])
np.linalg.inv(A)

array([[0.33333333, 0.66666667],
       [0.66666667, 0.33333333]])

In [8]:
(g_ba @ V_a_top_b).round(decimals=3)

array([[1.003, 0.997],
       [0.003, 0.007]])

In [9]:
np.linalg.inv(g_ba @ V_a_top_b).round(decimals=3)

array([[   1.743, -260.604],
       [  -0.749,  261.919]])

In [10]:
g_ab = V_a_top_b @ np.linalg.inv(g_ba @ V_a_top_b)
g_ab

array([[ 1.74502383e+00, -2.61261890e+02],
       [-7.45023827e-01,  2.61261890e+02],
       [-6.23545054e-17,  5.00000000e-01],
       [ 6.24464685e-17,  5.00000000e-01]])

In [11]:
g_ab_pinv = g_ba.T @ np.linalg.inv(g_ba @ g_ba.T)
g_ab_pinv

array([[0.5, 0. ],
       [0.5, 0. ],
       [0. , 0.5],
       [0. , 0.5]])

In [12]:
g_ab.sum(axis=0)

array([1., 1.])

### Finally, M_b:

In [13]:
M_b = g_ba @ M_a @ V_a_top_b @ np.linalg.inv(g_ba @ V_a_top_b)
M_b.round(decimals=3)

array([[ 1.100000e+01, -1.045028e+03],
       [ 2.000000e-02,  1.020000e+00]])

In [14]:
M_b @ M_b @ M_b @ np.array([[1, 1.]]).T

array([[-116571.53145429],
       [   -269.23673063]])

## M_c

In [15]:
g_cb = np.array([
    [1., 1.],
])

In [16]:
Lambda_col_b, V_b = np.linalg.eig(M_b)
Lambda_col_b, V_b.round(decimals=3)

(array([8.01005376, 4.01004155]),
 array([[1.   , 1.   ],
        [0.003, 0.007]]))

In [17]:
n_c, n_b = g_cb.shape

idx = Lambda_col_b.argsort()[-n_c:][::-1]
Lambda_col_b_top_c = Lambda_col_b[idx]  # top n_b eigenvalues
V_b_top_c = V_b[:,idx]           # corresponding eigenvectors

Lambda_col_b_top_c, V_b_top_c.round(decimals=3)

(array([8.01005376]),
 array([[1.   ],
        [0.003]]))

In [18]:
(g_cb @ V_b_top_c).round(decimals=3)

array([[1.003]])

In [19]:
np.linalg.inv(g_cb @ V_b_top_c).round(decimals=3)

array([[0.997]])

## Finally, M_c

In [20]:
M_c = g_cb @ M_b @ V_b_top_c @ np.linalg.inv(g_cb @ V_b_top_c)
M_c.round(decimals=3)

array([[8.01]])

# Evolving in time

## Initial Conditions

In [21]:
N_a_0 = np.array([[1., 0., 1., 0.]]).T
N_a_0

array([[1.],
       [0.],
       [1.],
       [0.]])

In [22]:
N_b_0 = g_ba @ N_a_0
N_b_0

array([[1.],
       [1.]])

In [23]:
N_c_0 = g_cb @ N_b_0
N_c_0

array([[2.]])

## Time Series N_a

In [24]:
N_a_t = [(np.linalg.matrix_power(M_a, t) @ N_a_0).round(decimals=3) for t in range(10)]
N_a_t


# What best subspace to choose?

# Strictly required:
# Additional restriction on V such that always get positive entries of M_b
# Only allocate cases from a coarse-grained region into the fine grained regions that it includes
# (i.e. N^b = g_ba N^a)
# g_ba^{-1} should start with g_ba.T, only gets to modify entries that have 1's in them, have each column sum to 1.

# Desired:
# So long-term trajectory stays the same.
# Don't stray too far... How far off will we be?

# Questions:
# Does 2nd largest eigenvalue need to be here? [Can't be there exactly...]

# Ideas:
# Split cases only using top 1 eigenvector
# Split cases using 2x2 sub-matrices within the coarse-grained region

[array([[1.],
        [0.],
        [1.],
        [0.]]),
 array([[8.02],
        [0.02],
        [1.02],
        [0.02]]),
 array([[64.251],
        [ 0.171],
        [ 1.111],
        [ 0.111]]),
 array([[514.663],
        [  1.34 ],
        [  1.767],
        [  0.767]]),
 array([[4122.488],
        [  10.544],
        [   6.953],
        [   5.953]]),
 array([[33021.364],
        [   83.635],
        [   48.412],
        [   47.412]]),
 array([[264502.917],
        [   666.548],
        [   380.42 ],
        [   379.42 ]]),
 array([[2118682.628],
        [   5325.485],
        [   3039.713],
        [   3038.713]]),
 array([[16970761.886],
        [   42602.807],
        [   24340.579],
        [   24339.579]]),
 array([[1.35936716e+08],
        [3.41031675e+05],
        [1.94961027e+05],
        [1.94960027e+05]])]

## Time Series N_b

In [25]:
N_b_t = [(g_ba @ np.linalg.matrix_power(M_a, t) @ N_a_0).round(decimals=3) for t in range(10)]
N_b_t

[array([[1.],
        [1.]]),
 array([[8.04],
        [1.04]]),
 array([[64.422],
        [ 1.222]]),
 array([[516.002],
        [  2.534]]),
 array([[4133.032],
        [  12.905]]),
 array([[33104.999],
        [   95.824]]),
 array([[265169.465],
        [   759.84 ]]),
 array([[2124008.113],
        [   6078.427]]),
 array([[17013364.692],
        [   48680.157]]),
 array([[1.36277747e+08],
        [3.89921054e+05]])]

## Time Series N_c

In [26]:
N_c_t = [(g_cb @ g_ba @ np.linalg.matrix_power(M_a, t) @ N_a_0).round(decimals=3) for t in range(10)]
N_c_t

[array([[2.]]),
 array([[9.08]]),
 array([[65.643]]),
 array([[518.537]]),
 array([[4145.937]]),
 array([[33200.822]]),
 array([[265929.305]]),
 array([[2130086.539]]),
 array([[17062044.85]]),
 array([[1.36667668e+08]])]

## Time Series N_b_hat

In [27]:
N_b_hat_t = [(np.linalg.matrix_power(M_b, t) @ N_b_0).round(decimals=3) for t in range(10)]
N_b_hat_t

[array([[1.],
        [1.]]),
 array([[-1034.027],
        [    1.04 ]]),
 array([[-12461.229],
        [   -19.62 ]]),
 array([[-116571.531],
        [   -269.237]]),
 array([[-1000938.153],
        [   -2606.052]]),
 array([[-8287018.818],
        [  -22676.936]]),
 array([[-67459973.542],
        [  -188870.851]]),
 array([[-5.44690894e+08],
        [-1.54184774e+06]]),
 array([[-4.38037836e+09],
        [-1.24665026e+07]]),
 array([[-3.51567407e+10],
        [-1.00323400e+08]])]

## Time Series N_c_hat

In [28]:
N_c_hat_t = [(np.linalg.matrix_power(M_c, t) @ N_c_0).round(decimals=3) for t in range(10)]
N_c_hat_t

[array([[2.]]),
 array([[16.02]]),
 array([[128.322]]),
 array([[1027.865]]),
 array([[8233.258]]),
 array([[65948.838]]),
 array([[528253.739]]),
 array([[4231340.843]]),
 array([[33893267.612]]),
 array([[2.71486896e+08]])]

## An alternate M_b construction

In [29]:
M_a

array([[8.01, 0.01, 0.01, 0.01],
       [0.01, 4.01, 0.01, 0.01],
       [0.01, 0.01, 1.01, 0.01],
       [0.01, 0.01, 0.01, 1.01]])

In [30]:
g_ba

array([[1., 1., 0., 0.],
       [0., 0., 1., 1.]])

In [31]:
g_ba @ M_a @ g_ba.T

array([[12.04,  0.04],
       [ 0.04,  2.04]])

In [32]:
g_ba @ M_a @ np.linalg.pinv(g_ba)

array([[6.02, 0.02],
       [0.02, 1.02]])

## Two Similar Matrices

In [33]:
M_a

array([[8.01, 0.01, 0.01, 0.01],
       [0.01, 4.01, 0.01, 0.01],
       [0.01, 0.01, 1.01, 0.01],
       [0.01, 0.01, 0.01, 1.01]])

In [34]:
g_ba

array([[1., 1., 0., 0.],
       [0., 0., 1., 1.]])

In [35]:
M_b = projector.sub_matrix_eigenvector(M_a, g_ba)
M_b

array([[8.010025, 0.02    ],
       [0.02    , 1.02    ]])

In [36]:
M_a[0, 1] = M_a[1, 0] = 0.1
M_a[2, 3] = M_a[3, 2] = 0.1
M_a[1, 3] = M_a[3, 1] = 0.05
M_a

array([[8.01, 0.1 , 0.01, 0.01],
       [0.1 , 4.01, 0.01, 0.05],
       [0.01, 0.01, 1.01, 0.1 ],
       [0.01, 0.05, 0.1 , 1.01]])

In [37]:
M_b = projector.sub_matrix_eigenvector(M_a, g_ba)
M_b

array([[8.01249844, 0.04      ],
       [0.02097502, 1.11      ]])